Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import folium

Load Data

In [2]:
df = pd.read_csv("data_psikologi_with_uuid.csv")

In [3]:
df_original = df.copy()

# Siapkan fitur
location_features = df[['latitude', 'longitude']].values
rating_review_features = df[['rating', 'review_count']].values

# Normalisasi lokasi & rating-review
location_scaler = MinMaxScaler()
rating_review_scaler = MinMaxScaler()

location_scaled = location_scaler.fit_transform(location_features)
rating_review_scaled = rating_review_scaler.fit_transform(rating_review_features)

# Gabungkan dan beri bobot
all_features_scaled = np.hstack([location_scaled, rating_review_scaled])
weights = np.array([0.4, 0.4, 0.1, 0.1])
weighted_features = all_features_scaled * weights

In [ ]:
def recommend_places(user_lat, user_long, df, weighted_features, location_scaler, rating_review_scaler, top_n=5):
    avg_rating = df['rating'].max()
    avg_review = df['review_count'].mean()

    user_loc_scaled = location_scaler.transform([[user_lat, user_long]])
    user_rat_scaled = rating_review_scaler.transform([[avg_rating, avg_review]])
    
    user_vector = np.hstack([user_loc_scaled, user_rat_scaled])[0] * weights

    similarity = cosine_similarity([user_vector], weighted_features)[0]
    df_result = df.copy()
    df_result['similarity'] = similarity

    top_result = df_result.sort_values(by='similarity', ascending=False).head(top_n).reset_index(drop=True)
    return top_result


In [ ]:
user_input = {
    'latitude': -6.9091778438223574, 
    'longitude': 107.8079707099244
}

top5 = recommend_places(user_input['latitude'], user_input['longitude'], df, weighted_features, location_scaler, rating_review_scaler)

print("Rekomendasi Teratas:")
display(top5)
print("ID dari rekomendasi:", top5['id'].tolist())

m = folium.Map(location=[user_input['latitude'], user_input['longitude']], zoom_start=15)
folium.Marker(
    location=[user_input['latitude'], user_input['longitude']],
    popup="Lokasi Anda",
    icon=folium.Icon(color='red', icon='user')
).add_to(m)

for _, row in top5.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']}<br>Similarity: {row['similarity']:.4f}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

m


Rekomendasi Teratas:


,id,name,category,addres_full,website,phone_number,rating,review_count,longitude,latitude,provinsi,similarity
0,e63beac6-3dcf-4862-8d51-5f745b2c6465,Darunnisa Psychological Services,Psychologist,"darunnisa psychological services, jl. cibiru h...",https://www.instagram.com/darunnisa,+62 819-1008-8911,5.0,12.0,107.722601,-6.936937,Jawa Barat,0.999989
1,24b21fbf-9f8f-4248-bf01-2649ea27c8e3,"Poli Psikologi, Rumah Sakit Harapan Keluarga (...",Psychologist,"poli psikologi, rumah sakit harapan keluarga (...",tidak diketahui,+62 821-3868-1319,5.0,5.0,107.762887,-6.949763,Jawa Barat,0.999981
2,44598d23-1cd8-4d8b-ac42-1f70badec471,Insight Psikotes,Psychologist,"insight psikotes, jl. antanila i no.9, cisaran...",tidak diketahui,+62 853-2050-3038,5.0,2.0,107.679313,-6.935846,Jawa Barat,0.999980
3,a07baea2-40c7-41a0-879f-d532a38b3769,Biro Psikologi Psy Center,Psychologist,"biro psikologi psy center, jl. purwakarta no.1...",tidak diketahui,+62 813-2200-1987,5.0,12.0,107.656931,-6.917371,Jawa Barat,0.999980
4,8470b50e-861b-4487-9c1c-1513bc86f6ab,Psikolog Sumedang,Psychologist,tidak diketahui,tidak diketahui,+62 812-2171-8476,5.0,18.0,107.907972,-6.853597,Jawa Barat,0.999977


ID dari rekomendasi: ['e63beac6-3dcf-4862-8d51-5f745b2c6465', '24b21fbf-9f8f-4248-bf01-2649ea27c8e3', '44598d23-1cd8-4d8b-ac42-1f70badec471', 'a07baea2-40c7-41a0-879f-d532a38b3769', '8470b50e-861b-4487-9c1c-1513bc86f6ab']
